# Convertir los datos diarios de la carpeta POR_VARIABLE a mensuales

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
'''rango de fechas'''
fecha_inicio = '1981-01-01'
fecha_fin = '2020-12-31'

'''leer los archivos en un carpeta dada SI son archivos CSV'''
def leer_dir(path):
    with os.scandir(path) as ficheros:
        ficheros = [fichero.name for fichero in ficheros if fichero.is_file() 
                    and fichero.name.endswith('.csv') 
                    and fichero.name.count('diario')>0]
    ficheros.sort()
    return ficheros

def leer_csv(file, path = '../datos/por_variables/'):
    p = path+file
    data = pd.read_csv(p)
    if 'Unnamed: 0' in data.columns:
        data.rename(columns={'Unnamed: 0':'fecha'}, inplace = True)
    return data  

'''convertir cadena a formato fecha'''
def conv_fecha(data, fec = 'fecha'):
    data[fec] = pd.to_datetime(data[fec])
    return data

def rango_fecha(data):
    data = data[(data['fecha'] >= fecha_inicio) & (data['fecha'] <= fecha_fin)]
    data.reset_index(drop = True, inplace = True)
    return data

'''convertir a DF mensual con la suma'''
def conv_mensual_suma(data):
    data = conv_fecha(data)
    data = rango_fecha(data)
    '''el pararmetro <min_count=1> mantiene los valores NaN'''
    data = data.groupby(pd.Grouper(key = 'fecha', freq = 'MS')).sum(min_count=1).reset_index()
    return data

'''convertir a DF mensual con el promedio'''
def conv_mensual_media(data):
    data = conv_fecha(data)
    data = rango_fecha(data)
    data = data.groupby(pd.Grouper(key = 'fecha', freq = 'MS')).mean().reset_index()
    return data

def guardar(data, nombre, path = '../datos/por_variables/'):
    p = path + nombre
    data.to_csv(p, index=False)

In [3]:
'''leer archivos CSV del directorio'''
lista_arch = leer_dir('../datos/por_variables/')
lista_arch

['chirps_diario_pp.csv',
 'gmet_diario_pp.csv',
 'gmet_diario_tmed.csv',
 'nasa_diario_hmed.csv',
 'nasa_diario_pp.csv',
 'nasa_diario_tmax.csv',
 'nasa_diario_tmed.csv',
 'nasa_diario_tmin.csv',
 'obs_diario_hmax.csv',
 'obs_diario_hmed.csv',
 'obs_diario_hmin.csv',
 'obs_diario_pp.csv',
 'obs_diario_tmax.csv',
 'obs_diario_tmed.csv',
 'obs_diario_tmin.csv']

In [4]:
for archivo in lista_arch:
    df_estacion = leer_csv(archivo)
    if archivo.count('pp') > 0:
        dfres = conv_mensual_suma(df_estacion)
    else:
        dfres = conv_mensual_media(df_estacion)
    archivo = archivo.replace('diario','mensual')
    guardar(dfres, archivo)
